In [1]:
import sys
import os
import torch



# Add the path to stylegan2-ada-pytorch directory
repo_path = '/Users/oliverlundin/Local Documents/github/facemorph/stylegan2-ada-pytorch'
sys.path.append(repo_path)

import projector



In [2]:
face1_path = 'face1.png'
face2_path = 'face2.png'

In [3]:
import torch
from PIL import Image
import numpy as np

face1 = Image.open(face1_path)
face2 = Image.open(face2_path)

# Convert the PIL images to NumPy arrays
face1_array = np.array(face1)
face2_array = np.array(face2)

# Convert NumPy arrays to PyTorch tensors with specified dtype
face1_tensor = torch.tensor(face1_array, dtype=torch.float32)  # Change dtype as needed
face2_tensor = torch.tensor(face2_array, dtype=torch.float32)

# Print the shape of the tensors
print(face1_tensor.shape)
print(face2_tensor.shape)


torch.Size([512, 512, 3])
torch.Size([512, 512, 3])


In [4]:
import pickle
import torch.nn.functional as F


# Set device
if(torch.backends.mps.is_available()): # True
	device = torch.device("mps")

# Load the generator model from the pickle file
with open('ffhq.pkl', 'rb') as f:
    G = pickle.load(f)['G_ema']
    

G.to(device)

print(f"Image channels: {G.img_channels}, Image resolution: {G.img_resolution}")

face1_tensor = face1_tensor.squeeze()
face1_tensor = face1_tensor.permute(2, 0, 1)
face1_tensor = F.interpolate(face1_tensor.unsqueeze(0), size=(G.img_resolution, G.img_resolution), mode='bilinear', align_corners=False)
face1_tensor = face1_tensor.squeeze(0)

face2_tensor = face2_tensor.squeeze()
face2_tensor = face2_tensor.permute(2, 0, 1)
face2_tensor = F.interpolate(face2_tensor.unsqueeze(0), size=(G.img_resolution, G.img_resolution), mode='bilinear', align_corners=False)
face2_tensor = face2_tensor.squeeze(0)

face1_tensor = face1_tensor.to(device)
face2_tensor = face2_tensor.to(device)

# Debugging: Print the shapes
print(f"face1_tensor shape: {face1_tensor.shape}")
print(f"face2_tensor shape: {face2_tensor.shape}")
print(f"Expected shape: ({G.img_channels}, {G.img_resolution}, {G.img_resolution})")



Image channels: 3, Image resolution: 1024
face1_tensor shape: torch.Size([3, 1024, 1024])
face2_tensor shape: torch.Size([3, 1024, 1024])
Expected shape: (3, 1024, 1024)


In [5]:

# Project the image
projected_w_steps1 = projector.project(
    G,
    target=face1_tensor,  # Your target image tensor
    num_steps=50,  # Number of optimization steps
    device=device,
    verbose=True  # Print optimization progress
)


Computing W midpoint and stddev using 10000 samples...
step    1/50: dist 0.60 loss 26969.62
step    2/50: dist 0.61 loss 67702.83
step    3/50: dist 0.58 loss 59981.11
step    4/50: dist 0.50 loss 45369.52
step    5/50: dist 0.50 loss 33338.46
step    6/50: dist 0.47 loss 22501.75
step    7/50: dist 0.42 loss 15647.18
step    8/50: dist 0.43 loss 11920.53
step    9/50: dist 0.40 loss 8755.54
step   10/50: dist 0.40 loss 6729.27
step   11/50: dist 0.44 loss 5830.00
step   12/50: dist 0.34 loss 4896.45
step   13/50: dist 0.35 loss 3967.25
step   14/50: dist 0.33 loss 3462.77
step   15/50: dist 0.31 loss 3050.70
step   16/50: dist 0.32 loss 2418.59
step   17/50: dist 0.32 loss 1878.26
step   18/50: dist 0.30 loss 1683.52
step   19/50: dist 0.29 loss 1724.79
step   20/50: dist 0.29 loss 1878.25
step   21/50: dist 0.30 loss 2115.12
step   22/50: dist 0.29 loss 2312.22
step   23/50: dist 0.29 loss 2301.37
step   24/50: dist 0.28 loss 2038.61
step   25/50: dist 0.28 loss 1599.96
step   26/50

In [6]:
projected_w_steps2 = projector.project(
    G,
    target=face2_tensor,  # Your target image tensor
    num_steps=50,  # Number of optimization steps
    device=device,
    verbose=True  # Print optimization progress
)

Computing W midpoint and stddev using 10000 samples...
step    1/50: dist 0.62 loss 2908.40
step    2/50: dist 0.54 loss 4715.89
step    3/50: dist 0.59 loss 3453.03
step    4/50: dist 0.55 loss 1272.85
step    5/50: dist 0.53 loss 5440.22
step    6/50: dist 0.51 loss 5022.19
step    7/50: dist 0.48 loss 3673.22
step    8/50: dist 0.47 loss 2854.46
step    9/50: dist 0.47 loss 1501.33
step   10/50: dist 0.47 loss 1166.68
step   11/50: dist 0.44 loss 1931.36
step   12/50: dist 0.43 loss 2124.79
step   13/50: dist 0.45 loss 1565.36
step   14/50: dist 0.42 loss 1173.43
step   15/50: dist 0.45 loss 1135.63
step   16/50: dist 0.45 loss 1009.88
step   17/50: dist 0.43 loss 775.13
step   18/50: dist 0.42 loss 739.83
step   19/50: dist 0.41 loss 842.73
step   20/50: dist 0.41 loss 787.93
step   21/50: dist 0.40 loss 611.14
step   22/50: dist 0.40 loss 532.31
step   23/50: dist 0.40 loss 517.55
step   24/50: dist 0.40 loss 437.93
step   25/50: dist 0.39 loss 332.23
step   26/50: dist 0.39 loss 

In [ ]:
projected_w_steps1.shape
projected_w_steps2.shape

In [ ]:
# check if the projected_w_steps1 and projected_w_steps2 are exactly the same
(projected_w_steps1 == projected_w_steps2).all()

In [7]:
torch.mps.empty_cache()
w1 = projected_w_steps1

img1 = G.synthesis(w1, noise_mode='const', force_fp32=True)

img1 = (img1.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()

#show the image
import matplotlib.pyplot as plt
plt.imshow(img1)


RuntimeError: MPS backend out of memory (MPS allocated: 7.65 GB, other allocations: 9.30 GB, max allowed: 18.13 GB). Tried to allocate 1.56 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [9]:
w2 = projected_w_steps2
img2 = G.synthesis(w2, noise_mode='const', force_fp32=True)
img2 = (img2.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)[0].cpu().numpy()


In [ ]:
plt.imshow(img2)